# Problem set 3: Text analysis of DOJ press releases

**Total points (without extra credit)**: 52 

- For background:

    - DOJ is the federal law enforcement agency responsible for federal prosecutions; this contrasts with the local prosecutions in the Cook County dataset we analyzed earlier. Here's a short explainer on which crimes get prosecuted federally versus locally: https://www.criminaldefenselawyer.com/resources/criminal-defense/federal-crime/state-vs-federal-crimes.htm#:~:text=Federal%20criminal%20prosecutions%20are%20handled,of%20state%20and%20local%20law. 
    - Here's the Kaggle that contains the data: https://www.kaggle.com/jbencina/department-of-justice-20092018-press-releases 
    - Here's the code the dataset creator used to scrape those press releases here if you're interested: https://github.com/jbencina/dojreleases

## 0.0 Import packages

In [102]:
## helpful packages
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import random
import re
import string

## nltk imports
import nltk
### uncomment and run these lines if you haven't downloaded relevant nltk add-ons yet
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk import pos_tag
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

## spacy imports
import spacy
### uncomment and run the below line if you haven't loaded the en_core_web_sm library yet
! python -m spacy download en_core_web_sm
import en_core_web_sm
nlp = en_core_web_sm.load()

## vectorizer
from sklearn.feature_extraction.text import CountVectorizer

## sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## lda
from gensim import corpora
import gensim

## repeated printouts and wide-format text
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', None)

from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import pandas as pd
from collections import Counter

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/grayhemans/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/grayhemans/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 14.7 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


## 0.1 Load and clean text data

In [103]:
## first, unzip the file pset3_inputdata.zip 
## then, run this code to load the unzipped json file and convert to a dataframe
## (may need to change the pathname depending on where you store stuff)
## and convert some of the attributes from lists to values
doj = pd.read_json("combined.json", lines = True)

## due to json, topics are in a list so remove them and concatenate with ;
doj['topics_clean'] = ["; ".join(topic) 
                      if len(topic) > 0 else "No topic" 
                      for topic in doj.topics]

## similarly with components
doj['components_clean'] = ["; ".join(comp) 
                           if len(comp) > 0 else "No component" 
                           for comp in doj.components]

## drop older columns from data
doj = doj[['id', 'title', 'contents', 'date', 'topics_clean', 
           'components_clean']].copy()

doj.head()

id  \
0     None   
1  12-919    
2  11-1002   
3   10-015   
4   18-898   

                                                                                                          title  \
0                                                                  Convicted Bomb Plotter Sentenced to 30 Years   
1                              $1 Million in Restitution Payments Announced to Preserve North Carolina Wetlands   
2                 $1 Million Settlement Reached for Natural Resource Damages at Superfund Site in Massachusetts   
3                                          10 Las Vegas Men Indicted \r\nfor Falsifying Vehicle Emissions Tests   
4  $100 Million Settlement Will Speed Cleanup Work at Centredale Manor Superfund Site in North Providence, R.I.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

## 1. Tagging and sentiment scoring (17 points)

Focus on the following press release: `id` == "17-1204" about this pharmaceutical kickback prosecution: https://www.forbes.com/sites/michelatindera/2017/11/16/fentanyl-billionaire-john-kapoor-to-plead-not-guilty-in-opioid-kickback-case/?sh=21b8574d6c6c 

The `contents` column is the one we're treating as a document. You may need to to convert it from a pandas series to a single string.

We'll call the raw string of this press release `pharma`

In [3]:
## your code to subset to one press release and take the string

pharma_release = doj[doj['id'] == "17-1204"]

# Extract the contents of the press release
pharma_contents = pharma_release['contents'].iloc[0]

pharma_contents

'The founder and majority owner of Insys Therapeutics Inc., was arrested today and charged with leading a nationwide conspiracy to profit by using bribes and fraud to cause the illegal distribution of a Fentanyl spray intended for cancer patients experiencing breakthrough pain.\xa0"More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids. And yet some medical professionals would rather take advantage of the addicts than try to help them," said Attorney General Jeff Sessions. "This Justice Department will not tolerate this.\xa0 We will hold accountable anyone – from street dealers to corporate executives -- who illegally contributes to this nationwide epidemic.\xa0 And under the leadership of President Trump, we are fully committed to defeating this threat to the American people.”John N. Kapoor, 74, of Phoenix, Ariz., a current member of the Board of Directors of Insys, was arrested this morning in Arizona and charged with RICO conspi

### 1.1 part of speech tagging (3 points)

A. Preprocess the `pharma` press release to remove all punctuation / digits (you can use `.isalpha()` to subset)

B. With the preprocessed press release from part A, use the part of speech tagger within nltk to tag all the words in that one press release with their part of speech. 

C. Using the output from B, extract the adjectives and sort those adjectives from most occurrences to fewest occurrences. Print a dataframe with the 5 most frequent adjectives and their counts in the `pharma` release. See here for a list of the names of adjectives within nltk: https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/

**Resources**:

- Documentation for `.isalpha()`: https://www.w3schools.com/python/ref_string_isalpha.asp

In [4]:
## your code here to restrict to alpha

pharma_preprocessed = ''.join(char for char in pharma_contents if char.isalpha() or char.isspace())


In [5]:
## your code here for part of speech tagging

tokens = word_tokenize(pharma_preprocessed) # Generate list of tokens
tokens_pos = pos_tag(tokens) # generate part of speech tags for those tokens
tokens_pos

[('The', 'DT'),
 ('founder', 'NN'),
 ('and', 'CC'),
 ('majority', 'NN'),
 ('owner', 'NN'),
 ('of', 'IN'),
 ('Insys', 'NNP'),
 ('Therapeutics', 'NNP'),
 ('Inc', 'NNP'),
 ('was', 'VBD'),
 ('arrested', 'VBN'),
 ('today', 'NN'),
 ('and', 'CC'),
 ('charged', 'VBN'),
 ('with', 'IN'),
 ('leading', 'VBG'),
 ('a', 'DT'),
 ('nationwide', 'JJ'),
 ('conspiracy', 'NN'),
 ('to', 'TO'),
 ('profit', 'VB'),
 ('by', 'IN'),
 ('using', 'VBG'),
 ('bribes', 'NNS'),
 ('and', 'CC'),
 ('fraud', 'NN'),
 ('to', 'TO'),
 ('cause', 'VB'),
 ('the', 'DT'),
 ('illegal', 'JJ'),
 ('distribution', 'NN'),
 ('of', 'IN'),
 ('a', 'DT'),
 ('Fentanyl', 'NNP'),
 ('spray', 'NN'),
 ('intended', 'VBD'),
 ('for', 'IN'),
 ('cancer', 'NN'),
 ('patients', 'NNS'),
 ('experiencing', 'VBG'),
 ('breakthrough', 'NN'),
 ('pain', 'NN'),
 ('More', 'JJR'),
 ('than', 'IN'),
 ('Americans', 'NNPS'),
 ('died', 'VBD'),
 ('of', 'IN'),
 ('synthetic', 'JJ'),
 ('opioid', 'NN'),
 ('overdoses', 'NNS'),
 ('last', 'JJ'),
 ('year', 'NN'),
 ('and', 'CC'),
 (

In [6]:
# All adjectives
all_adjectives = [one_tok[0] for one_tok in tokens_pos 
                if one_tok[1] == "JJS" or 
               one_tok[1] == "JJR" or
               one_tok[1] == "JJ"]

adjectives_count = Counter(all_adjectives)

adjectives_count

Counter({'former': 8,
         'opioid': 5,
         'nationwide': 4,
         'other': 3,
         'addictive': 3,
         'illegal': 2,
         'accountable': 2,
         'corporate': 2,
         'current': 2,
         'federal': 2,
         'powerful': 2,
         'potent': 2,
         'streetlevel': 2,
         'important': 2,
         'patient': 2,
         'public': 2,
         'greater': 2,
         'supervised': 2,
         'More': 1,
         'synthetic': 1,
         'last': 1,
         'medical': 1,
         'later': 1,
         'additional': 1,
         'several': 1,
         'various': 1,
         'many': 1,
         'fentanylbased': 1,
         'narcotic': 1,
         'intense': 1,
         'large': 1,
         'most': 1,
         'reluctant': 1,
         'prior': 1,
         'ongoing': 1,
         'utilized': 1,
         'acceptable': 1,
         'better': 1,
         'pharmaceutical': 1,
         'responsible': 1,
         'corrupt': 1,
         'total': 1,
         's

In [8]:
# Create a DataFrame with the adjectives and their counts
adjectives_df = pd.DataFrame(adjectives_count.items(), columns=['Adjective', 'Count'])

# Sort adjectives by count in descending order
adjectives_df = adjectives_df.sort_values(by='Count', ascending=False)

# Print the top 5 most frequent adjectives
print(adjectives_df.head(5))

     Adjective  Count
10      former      8
27      opioid      5
0   nationwide      4
31   addictive      3
9        other      3


## 1.2 named entity recognition (4 points)

A. Using the original `pharma` press release (so the one before stripping punctuation/digits), use spaCy to extract all named entities from the press release.

B. Print the unique named entities with the tag: `LAW`

In [7]:
pharma_nlp = nlp(pharma_contents)
# loop over the doc object
named_entities = [ent.text for ent in pharma_nlp.ents]

#extract unique named entities with the tag "LAW"
unique_law_entities = set([ent.text for ent in pharma_nlp.ents if ent.label_ == "LAW"])

C. Use Google to summarize in one sentence what the `RICO` named entity means and why this might apply to a pharmaceutical kickbacks case (and not just a mafia case...) 

Pharmaceutical companies may be subject to investigations and legal actions under the Racketeer Influenced and Corrupt Organizations Act (RICO) if they are alleged to have participated in activities that breach this law.

D. You want to extract the possible sentence lengths the CEO is facing; pull out the named entities with (1) the label `DATE` and (2) that contain the word year or years (hint: you may want to use the `re` module for that second part). Print these named entities.

In [10]:
# extract the named entities with the pos tag DATE
date_entities = [ent.text for ent in pharma_nlp.ents if ent.label_ == "DATE" and re.search(r'\b(?:year|years)\b', ent.text, re.I)]
date_entities

['last year', '20 years', 'three years', 'five years', 'three years']

E. Pull and print the original parts of the press releases where those year lengths are mentioned (e.g., the sentences or rough region of the press release). Describe in your own words (1 sentence) what length of sentence (prison) and probation (supervised release) the CEO may be facing if convicted after this indictment (if there are multiple lengths mentioned describe the maximum). 

**Hint**: you may want to use re.search or re.findall 

- For part E, you can use `re.search` and `re.findall`, or anything that works 😳.

In [83]:
# Define a regex pattern to find sentences mentioning year lengths
pattern = r'(?:\b\d+\s*(?:year|years)\s*(?:in\s*prison|in\s*custody|probation)?\b)'

matches = re.findall(pattern, pharma_contents)
matches

# If convicted the CEO may face 20 years in Prison. 

'(?:\\b\\d+\\s*(?:year|years)\\s*(?:in\\s*prison|in\\s*custody|probation)?\\b)'

['20 years in prison']

'The CEO, if convicted, may face up to 20 years in prison and might be eligible for five years on probation.'

## 1.3 sentiment analysis  (10 points)

A. Subset the press releases to those labeled with one of three topics via `topics_clean`: Civil Rights, Hate Crimes, and Project Safe Childhood. We'll call this `doj_subset` going forward and it should have 717 rows.



In [17]:
topics_of_interest = ['Civil Rights', 'Hate Crimes', 'Project Safe Childhood']

doj_subset = doj[doj['topics_clean'].isin(topics_of_interest)]

print("Number of rows in doj_subset:", len(doj_subset)) 
doj_subset.shape

Number of rows in doj_subset: 717


(717, 6)

B. Write a function that takes one press release string as an input and:

- Removes named entities from each press release string (**Hint**: you may want to use `re.sub` with an or condition)
- Scores the sentiment of the entire press release using the `SentimentIntensityAnalyzer` and `polarity_scores`
- Returns the length-four (negative, positive, neutral, compound) sentiment dictionary (any order is fine)

Apply that function to each of the press releases in `doj_subset`. 

**Hints**: 

- A function + list comprehension to execute will takes about 30 seconds on a respectable local machine and about 2 mins on jhub; if it's taking a very long time, you may want to check your code for inefficiencies. If you can't fix those, for partial credit on this part/full credit on remainder, you can take a small random sample of the 717


In [20]:
analyzer = SentimentIntensityAnalyzer()

def process_press_release(press_release):
    press_release_clean = re.sub(r'\b[A-Z][a-z]*\b', '', press_release)
    
    sentiment_scores = analyzer.polarity_scores(press_release_clean)
    
    return sentiment_scores

doj_subset['sentiment'] = doj_subset['contents'].apply(process_press_release)
print(doj_subset[['id', 'sentiment']])

            id  \
77     17-1235   
155    15-1522   
157     16-213   
162     16-381   
168     14-464   
...        ...   
13002   09-368   
13032   18-775   
13034   12-596   
13068   18-359   
13081  14-1377   

                                                             sentiment  
77       {'neg': 0.191, 'neu': 0.759, 'pos': 0.05, 'compound': -0.992}  
155      {'neg': 0.113, 'neu': 0.82, 'pos': 0.067, 'compound': -0.891}  
157     {'neg': 0.088, 'neu': 0.84, 'pos': 0.072, 'compound': -0.7579}  
162    {'neg': 0.122, 'neu': 0.796, 'pos': 0.082, 'compound': -0.9037}  
168    {'neg': 0.173, 'neu': 0.784, 'pos': 0.043, 'compound': -0.9864}  
...                                                                ...  
13002  {'neg': 0.145, 'neu': 0.792, 'pos': 0.063, 'compound': -0.9664}  
13032   {'neg': 0.078, 'neu': 0.831, 'pos': 0.092, 'compound': 0.5267}  
13034  {'neg': 0.153, 'neu': 0.769, 'pos': 0.078, 'compound': -0.9705}  
13068  {'neg': 0.134, 'neu': 0.771, 'pos': 0.095, 'co

In [85]:
doj_subset['sentiment'] = doj_subset['contents'].apply(process_press_release)
print(doj_subset[['id', 'sentiment']])

            id  \
77     17-1235   
155    15-1522   
157     16-213   
162     16-381   
168     14-464   
...        ...   
13002   09-368   
13032   18-775   
13034   12-596   
13068   18-359   
13081  14-1377   

                                                             sentiment  
77       {'neg': 0.191, 'neu': 0.759, 'pos': 0.05, 'compound': -0.992}  
155      {'neg': 0.113, 'neu': 0.82, 'pos': 0.067, 'compound': -0.891}  
157     {'neg': 0.088, 'neu': 0.84, 'pos': 0.072, 'compound': -0.7579}  
162    {'neg': 0.122, 'neu': 0.796, 'pos': 0.082, 'compound': -0.9037}  
168    {'neg': 0.173, 'neu': 0.784, 'pos': 0.043, 'compound': -0.9864}  
...                                                                ...  
13002  {'neg': 0.145, 'neu': 0.792, 'pos': 0.063, 'compound': -0.9664}  
13032   {'neg': 0.078, 'neu': 0.831, 'pos': 0.092, 'compound': 0.5267}  
13034  {'neg': 0.153, 'neu': 0.769, 'pos': 0.078, 'compound': -0.9705}  
13068  {'neg': 0.134, 'neu': 0.771, 'pos': 0.095, 'co

C. Add the four sentiment scores to the `doj_subset` dataframe to create a dataframe: `doj_subset_wscore`. Sort from highest neg to lowest neg score and print the top `id`, `contents`, and `neg` columns of the two most neg press releases. 

Notes:

- Don't worry if your sentiment score differs slightly from our output on GitHub; differences in preprocessing can lead to diff scores

In [19]:
# Part C

# parse the sentiment scores into seperate columns and add to doj_subset
doj_subset['negative'] = doj_subset['sentiment'].apply(lambda x: x['neg'])
doj_subset['neutral'] = doj_subset['sentiment'].apply(lambda x: x['neu'])
doj_subset['positive'] = doj_subset['sentiment'].apply(lambda x: x['pos'])
doj_subset['compound'] = doj_subset['sentiment'].apply(lambda x: x['compound'])

doj_subset_wscore = doj_subset[['id', 
                                'contents', 
                                'negative', 
                                'positive', 
                                'neutral', 
                                'compound',
                               'topics_clean']].sort_values(by='negative', ascending=False)

top_2_neg_press_releases = doj_subset_wscore.head(2)
top_2_neg_press_releases[['id', 'contents', 'negative', 'neutral', 'positive', 'compound']]

,id,contents,negative,neutral,positive,compound
329,14-248,"The Department of Justice announced that this morning John W. Ng, 58, of Albuquerque, N.M., made his initial appearance in federal court on a criminal complaint charging him with a hate crime offense. This charge is related to anti-Semitic threats Ng made against a Jewish woman who owns and operates the Nosh Jewish Delicatessen and Bakery in Albuquerque. Ng was arrested by the FBI on March 7, 2014, based on a criminal complaint alleging that he interfered with the victim’s federally protected rights by threatening her and interfering with her business because of her religion. According to the criminal complaint, between Jan. 22, 2014, and Feb. 8, 2014, Ng allegedly posted threatening anti-Semitic notes on and in the vicinity of the victim’s business. A criminal complaint merely establishes probable cause, and Ng is presumed innocent unless proven guilty. If convicted on the offense charged in the criminal complaint, Ng faces a maximum statutory penalty of one year in prison. This matter was investigated by the Albuquerque Division of the FBI and is being prosecuted by Assistant U.S. Attorney Mark T. Baker of the U.S. Attorney’s Office for the District of New Mexico and Trial Attorney AeJean Cha of the U.S. Department of Justice’s Civil Rights Division.",0.321,0.650,0.029,-0.9957
11593,16-718,"In a nine-count indictment unsealed today, two Mississippi correctional officers were charged with beating an inmate and a third was charged with helping to cover it up. The indictment charged Lawardrick Marsher, 28, and Robert Sturdivant, 47, officers at Mississippi State Penitentiary, in Parchman, Mississippi, with a beating that included kicking, punching and throwing the victim to the ground. Marsher and Sturdivant were charged with violating the right of K.H., a convicted prisoner, to be free from cruel and unusual punishment. Sturdivant was also charged with failing to intervene while Marsher was punching and beating K.H. The indictment alleges that their actions involved the use of a dangerous weapon and resulted in bodily injury to the victim. A third officer, Deonte Pate, 23, was charged along with Marsher and Sturdivant for conspiring to cover up the beating. The indictment alleges that all three officers submitted false reports and that all three lied to the FBI. If convicted, Marsher and Sturdivant face a maximum sentence of 10 years in prison on the excessive force charges. Each of the three officers faces up to five years in prison on the conspiracy and false statement charges, and up to 20 years in prison on the false report charges. An indictment is merely an accusation, and the defendants are presumed innocent unless and until proven guilty. This case is being investigated by the FBI’s Jackson Division, with the cooperation of the Mississippi Department of Corrections. It is being prosecuted by Assistant U.S. Attorney Robert Coleman of the Northern District of Mississippi and Trial Attorney Dana Mulhauser of the Civil Rights Division’s Criminal Section. Marsher Indictment",0.289,0.679,0.032,-0.9964


D. With the dataframe from part C, find the mean compound sentiment score for each of the three topics in `topics_clean` using group_by and agg.

E. Add a 1 sentence interpretation of why we might see the variation in scores (remember that compound is a standardized summary where -1 is most negative; +1 is most positive)

In [23]:
## agg and find the mean compound score by topic
mean_compound_score = doj_subset_wscore.groupby('topics_clean')['compound'].agg('mean')

mean_compound_score

topics_clean
Civil Rights             -0.106531
Hate Crimes              -0.934456
Project Safe Childhood   -0.722572
Name: compound, dtype: float64

The variation in sentiment scores is due to the variation of each topic -- it is appropriate that hate crimes are very negatively discussed, and civil rights and Project Safe Childhood are discussed less negatively.

# 2. Topic modeling (25 points)

For this question, use the `doj_subset_wscores` data that is restricted to civil rights, hate crimes, and project safe childhood and with the sentiment scores added

## 2.1 Preprocess the data by removing stopwords, punctuation, and non-alpha words (5 points)

A. Write a function that:

- Takes in a single raw string in the `contents` column from that dataframe
- Does the following preprocessing steps:

    - Converts the words to lowercase
    - Removes stopwords, adding the custom stopwords in your code cell below to the default stopwords list
    - Only retains alpha words (so removes digits and punctuation)
    - Only retains words 4 characters or longer
    - Uses the snowball stemmer from nltk to stem

- Returns a joined preprocessed string
    
B. Use `apply` or list comprehension to execute that function and create a new column in the data called `processed_text`
    
C. Print the `id`, `contents`, and `processed_text` columns for the following press releases:

id = 16-718 (this case: https://www.seattletimes.com/nation-world/doj-miami-police-reach-settlement-in-civil-rights-case/)

id = 16-217 (this case: https://www.wlbt.com/story/32275512/three-mississippi-correctional-officers-indicted-for-inmate-assault-and-cover-up/)
    
**Resources**:

- Here's code examples for the snowball stemmer: https://www.geeksforgeeks.org/snowball-stemmer-nlp/

In [104]:
custom_stopwords = ["civil", "rights", "division", "department", "justice",
                        "office", "attorney", "district", "case", "investigation", "assistant",
                       "trial", "assistance", "assist"]

In [105]:
def preprocess_text(text, custom_stopwords):
    text = text.lower()

    words = word_tokenize(text)

    stop_words = set(stopwords.words('english'))
    stop_words.update(custom_stopwords)

    stemmer = SnowballStemmer('english')
    
    processed_words = [
        stemmer.stem(word) for word in words
        if word.isalpha() and
        word not in stop_words
        and len(word) >= 4
    ]

    return ' '.join(processed_words)

In [106]:
doj_subset_wscore['processed_text'] = doj_subset_wscore['contents'].apply(lambda x: preprocess_text(x, custom_stopwords))

AttributeError: 'float' object has no attribute 'lower'

In [ ]:
## your code showing the examples
miami_case = doj_subset_wscore[doj_subset_wscore['id'] == '16-718']
miami_case[['id', 'contents', 'processed_text']]

missisippi_case = doj_subset_wscore[doj_subset_wscore['id'] == '16-217']
missisippi_case[['id', 'contents', 'processed_text']]

## 2.2 Create a document-term matrix from the preprocessed press releases and to explore top words (5 points)

A. Use the `create_dtm` function I provide (alternately, feel free to write your own!) and create a document-term matrix using the preprocessed press releases; make sure metadata contains the following columns: `id`, `compound` sentiment column you added, and the `topics_clean` column

B. Print the top 10 words for press releases with compound sentiment in the top 5% (so the most positive sentiment)

C. Print the top 10 words for press releases with compound sentiment in the bottom 5% (so the most negative sentiment)

**Hint**: for these, remember the pandas quantile function from pset one.  

D. Print the top 10 words for press releases in each of the three `topics_clean`

For steps B - D, to receive full credit, write a function `get_topwords` that helps you avoid duplicated code when you find top words for the different subsets of the data. There are different ways to structure it but one way is to feed it subsetted data (so data subsetted to one topic etc.) and for it to get the top words for that subset.


In [99]:
def create_dtm(list_of_strings, metadata):
    vectorizer = CountVectorizer(lowercase = True)
    dtm_sparse = vectorizer.fit_transform(list_of_strings)
    dtm_dense_named = pd.DataFrame(dtm_sparse.todense(), 
        columns=vectorizer.get_feature_names_out())
    dtm_dense_named_withid = pd.concat([metadata.reset_index(), dtm_dense_named], axis = 1)
    return(dtm_dense_named_withid)

In [100]:
dtm = create_dtm(doj_subset_wscore['processed_text'], metadata = doj_subset_wscore[['id', 'compound', 'topics_clean']])

def get_topwords(dtm, column_name = 'processed_text', n_top = 10):
    text_concat = ' '.join(dtm[column_name])
    words = text_concat.split()
    word_counts = Counter(words)
    top_words = word_counts.most_common(n_top)
    return top_words

# Part B
top_5_percent_threshold = doj_subset_wscore['compound'].quantile(0.95)
top_5_percent_dtm = doj_subset_wscore[doj_subset_wscore['compound'] >= top_5_percent_threshold]
top_5_percent_positive_words = get_topwords(top_5_percent_dtm)
top_5_percent_positive_words

# Part C
bottom_5_percent_threshold = doj_subset_wscore['compound'].quantile(0.05)
bottom_5_percent_dtm = doj_subset_wscore[doj_subset_wscore['compound'] <= bottom_5_percent_threshold]
bottom_5_percent_positive_words = get_topwords(bottom_5_percent_dtm)
bottom_5_percent_positive_words

# Part D
unique_topics = doj_subset_wscore['topics_clean'].unique()
for topic in unique_topics:
    topic_dtm = doj_subset_wscore[doj_subset_wscore['topics_clean'] == topic]
    top_words = get_topwords(topic_dtm)
    print(f"Top 10 words for topic '{topic}':")
    top_words

[('agreement', 176),
 ('state', 123),
 ('enforc', 116),
 ('ensur', 105),
 ('disabl', 102),
 ('polic', 88),
 ('settlement', 88),
 ('student', 86),
 ('general', 83),
 ('communiti', 82)]

[('assault', 202),
 ('victim', 185),
 ('crime', 168),
 ('offic', 164),
 ('hate', 135),
 ('defend', 130),
 ('sentenc', 111),
 ('feder', 101),
 ('charg', 101),
 ('prosecut', 97)]

Top 10 words for topic 'Hate Crimes':


[('victim', 590),
 ('crime', 533),
 ('prosecut', 476),
 ('hate', 472),
 ('defend', 459),
 ('sentenc', 455),
 ('charg', 452),
 ('guilti', 430),
 ('feder', 426),
 ('said', 424)]

Top 10 words for topic 'Civil Rights':


[('offic', 627),
 ('hous', 620),
 ('discrimin', 541),
 ('enforc', 531),
 ('disabl', 509),
 ('said', 497),
 ('feder', 475),
 ('violat', 470),
 ('state', 443),
 ('general', 408)]

Top 10 words for topic 'Project Safe Childhood':


[('child', 1018),
 ('exploit', 698),
 ('sexual', 570),
 ('safe', 476),
 ('project', 472),
 ('childhood', 472),
 ('pornographi', 447),
 ('children', 416),
 ('crimin', 404),
 ('prosecut', 374)]

Top 10 words for topic 'nan':


[]

In [108]:
def create_dtm(texts, metadata):
    vectorizer = CountVectorizer(lowercase=True)
    dtm_sparse = vectorizer.fit_transform(texts)
    dtm_dense = pd.DataFrame(dtm_sparse.todense(), columns=vectorizer.get_feature_names_out())
    # Concatenate using a proper index reset for both metadata and dtm_dense to avoid any indexing issues
    dtm_final = pd.concat([metadata.reset_index(drop=True), dtm_dense.reset_index(drop=True)], axis=1)
    return dtm_final

# Main script setup
metadata = doj_subset_wscore[['id', 'compound', 'topics_clean']]
text = doj_subset_wscore['processed_text']
dtm = create_dtm(text, metadata)

# Calculate the top 5% quantile for compound sentiment score
top_5_percentile = dtm['compound'].quantile(0.95)
# Filter the DTM for rows where 'compound' is greater than or equal to the top 5% quantile
top_words_positive = get_top_words(dtm[dtm['compound'] >= top_5_percentile])

# Calculate the bottom 5% quantile for compound sentiment score
bottom_5_percentile = dtm['compound'].quantile(0.05)
# Filter the DTM for rows where 'compound' is less than or equal to the bottom 5% quantile
top_words_negative = get_top_words(dtm[dtm['compound'] <= bottom_5_percentile])

# Display results
print("Top 10 words in the most positive press releases:\n", top_words_positive)
print("Top 10 words in the most negative press releases:\n", top_words_negative)

# Analyze top words by topic
topics = dtm['topics_clean'].unique()
topic_words = {}
for topic in topics:
    topic_dtm = dtm[dtm['topics_clean'] == topic]
    topic_words[topic] = get_top_words(topic_dtm)
    print(f"\nTop 10 words for '{topic}':\n", topic_words[topic])





ValueError: cannot reindex on an axis with duplicate labels

## 2.3 Estimate a topic model using those preprocessed words (5 points)

A. Going back to the preprocessed words from part 2.3.1, estimate a topic model with 3 topics, since you want to see if the unsupervised topic models recover different themes for each of the three manually-labeled areas (civil rights; hate crimes; project safe childhood). You have free rein over the other topic model parameters beyond the number of topics.

B. After estimating the topic model, print the top 15 words in each topic.

**Hints and Resources**:

- Same topic modeling resources linked to above
- Make sure to use the `random_state` argument within the model so that the numbering of topics does not move around between runs of your code

In [67]:
selected_topics = ['Civil Rights', 'Hate Crimes', 'Project Safe Childhood']

filtered_df = doj_subset_wscore[doj_subset_wscore['topics_clean'].isin(selected_topics)]

tokenized_texts = [wordpunct_tokenize(text) for text in filtered_df.processed_text]

dictionary = corpora.Dictionary(tokenized_texts)
initial_dictionary_size = len(dictionary)  # Store initial size for comparison

doc_count = filtered_df.shape[0]  # Number of documents
lower_threshold = round(doc_count * 0.023)
upper_threshold = round(doc_count * 0.977) # thresholds make this 2 SDs from the mean

dictionary.filter_extremes(no_below = lower_threshold, no_above = upper_threshold)
filtered_dictionary_size = len(dictionary)  # Store size after filtering

print(f'Filtering reduced the dictionary size from {initial_dictionary_size} to {filtered_dictionary_size}.')

corpus = [dictionary.doc2bow(text) for text in tokenized_texts]

Filtering reduced the dictionary size from 6756 to 1151.


In [31]:
lda_model = gensim.models.ldamodel.LdaModel(
    corpus = corpus,
    num_topics = 3,
    id2word = dictionary,
    passes = 10,
    alpha = 'auto',
    per_word_topics = True,
    random_state = 100 # ensures reproducability
)

topics = lda_model.print_topics(num_words = 15)

In [32]:
for index, topic in enumerate(topics):
    words = [word.split("*")[1].strip(' "') for word in topic[1].split(" + ")]
    print(f"Topic {index + 1} top 15 words: {words}")        

Topic 1 top 15 words: ['victim', 'charg', 'sentenc', 'prosecut', 'defend', 'feder', 'guilti', 'crime', 'said', 'indict', 'hate', 'prison', 'investig', 'year', 'assault']
Topic 2 top 15 words: ['child', 'exploit', 'sexual', 'safe', 'project', 'childhood', 'pornographi', 'children', 'crimin', 'prosecut', 'victim', 'sentenc', 'ceo', 'minor', 'abus']
Topic 3 top 15 words: ['hous', 'discrimin', 'disabl', 'enforc', 'agreement', 'state', 'said', 'court', 'alleg', 'requir', 'feder', 'settlement', 'violat', 'fair', 'general']


## 2.4 Add topics back to main data and explore correlation between manual labels and our estimated topics (10 points)

A. Extract the document-level topic probabilities. Within `get_document_topics`, use the argument `minimum_probability` = 0 to make sure all 3 topic probabilities are returned. Write an assert statement to make sure the length of the list is equal to the number of rows in the `doj_subset_wscores` dataframe

B. Add the topic probabilities to the `doj_subset_wscores` dataframe as columns and create a column, `top_topic`, that reflects each document to its highest-probability topic (eg topic 1, 2, or 3)

C. For each of the manual labels in `topics_clean` (Hate Crime, Civil Rights, Project Safe Childhood), print the breakdown of the % of documents with each top topic (so, for instance, Hate Crime has 246 documents-- if 123 of those documents are coded to topic_1, that would be 50%; and so on). **Hint**: pd.crosstab and normalize may be helpful: https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.crosstab.html

D. Using a couple press releases as examples, write a 1-2 sentence interpretation of why some of the manual topics map on more cleanly to an estimated topic than other manual topic(s)

In [40]:
topic_probabilities = [lda_model.get_document_topics(item, minimum_probability = 0) 
                       for item in corpus
]

assert len(topic_probabilities) == doj_subset_wscore.shape[0]
topic_probabilities[0:5]

[[(0, 0.99864405), (1, 0.00060329743), (2, 0.0007526972)],
 [(0, 0.9991564), (1, 0.00037534026), (2, 0.00046828896)],
 [(0, 0.9992957), (1, 0.0003133395), (2, 0.00039093444)],
 [(0, 0.9993539), (1, 0.00028743627), (2, 0.00035861655)],
 [(0, 0.9988445), (1, 0.00051409745), (2, 0.0006414078)]]

In [35]:
### your code here to add those topic probabilities to the dataframe
### Intialize columns
#for i in range(3):
    #doj_subset_wscore[f'topic_{i + 1}_prob'] = 0

## Iterate over each document's topic probabilities
#for index, probabilities in enumerate(topic_probabilities):
    #for topic_id, prob in probabilities:
        #doj_subset_wscore.at[index, f'topic_{topic_id + 1}_prob'] = prob

    #max_topic = max(probabilities, key=lambda x: x[1])

    #doj_subset_wscore.at[index, 'top_topic'] = max_topic[0]


In [41]:
for i in range(3):
    doj_subset_wscore[f'topic_{i}_prob'] = 0

# Iterate over each document's topic probabilities
for index, probabilities in enumerate(topic_probabilities):
    for topic_id, prob in probabilities:
        doj_subset_wscore.at[index, f'topic_{topic_id}_prob'] = prob

    max_topic = max(probabilities, key=lambda x: x[1])
    
    doj_subset_wscore.at[index, 'top_topic'] = max_topic[0] + 1 

In [44]:
##C. For each of the manual labels in `topics_clean` 
#(Hate Crime, Civil Rights, Project Safe Childhood), print the breakdown of the 
#% of documents with each top topic (so, for instance, Hate Crime has 246 documents-- 
#if 123 of those documents are coded to topic_1, that would be 50%; and so on). 
#**Hint**: pd.crosstab and normalize may be helpful: https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.crosstab.html

topic_dist = pd.crosstab(doj_subset_wscore['topics_clean'], doj_subset_wscore['top_topic'], normalize='index')

topic_dist_percent = topic_dist * 100

topic_dist_percent

top_topic,1.0,2.0,3.0
topics_clean,,,
Civil Rights,33.333333,66.666667,0.000000
Hate Crimes,22.727273,50.000000,27.272727
Project Safe Childhood,63.636364,18.181818,18.181818


# 3. Extend the analysis from unigrams to bigrams (10 points)

In the previous question, you found top words via a unigram representation of the text. Now, we want to see how those top words change with bigrams (pairs of words)

A. Using the `doj_subset_wscore` data and the `processed_text` column (so the words after stemming/other preprocessing), create a column in the data called `processed_text_bigrams` that combines each consecutive pairs of word into a bigram separated by an underscore. Eg:

"depart reach settlem" would become "depart_reach reach_settlem"

Do this by writing a function `create_bigram_onedoc` that takes in a single `processed_text` string and returns a string with its bigrams structured similarly to above example
 
**Hint**: there are many ways to solve but `zip` may be helpful: https://stackoverflow.com/questions/21303224/iterate-over-all-pairs-of-consecutive-items-in-a-list



In [68]:
def create_bigram_onedoc(processed_text):
    if not isinstance(processed_text, str):
        return ""  

    words = processed_text.split()
    bigrams = ["_".join(pair) for pair in zip(words, words[1:])]
    return " ".join(bigrams)

doj_subset_wscore['processed_text_bigrams'] = doj_subset_wscore['processed_text'].apply(create_bigram_onedoc)

doj_subset_wscore[doj_subset_wscore['id'] == '16-217'][['id', 'processed_text', 'processed_text_bigrams']]

,id,processed_text,processed_text_bigrams
6727,16-217,reach comprehens settlement agreement citi miami miami polic resolv shoot offic announc princip deputi general vanita gupta head wifredo ferrer southern florida settlement approv miami citi commiss today effect agreement sign parti resolv claim stem shoot offic conduct violent crime control enforc find issu juli identifi pattern practic excess forc shoot violat fourth amend constitut citi complianc settlement monitor independ review former tampa florida polic chief jane castor settlement agreement citi implement comprehens reform ensur constitut polic support public trust settlement agreement design minim shoot effect quick investig shoot occur measur includ settlement repres renew commit citi miami chief rodolfo llane provid constitut polic miami resid protect public safeti sustain reform said princip deputi general gupta agreement help strengthen relationship communiti serv improv account offic fire weapon unlaw provid communiti particip enforc today agreement result joint effort citi miami ensur miami polic continu effort make communiti safe protect sacr constitut citizen said ferrer oversight communic agreement seek make perman posit chang former chief orosa chief llane made applaud citi commiss settlement agreement build upon import reform implement citi sinc issu find includ conduct attorney staff special litig section southern florida,reach_comprehens comprehens_settlement settlement_agreement agreement_citi citi_miami miami_miami miami_polic polic_resolv resolv_shoot shoot_offic offic_announc announc_princip princip_deputi deputi_general general_vanita vanita_gupta gupta_head head_wifredo wifredo_ferrer ferrer_southern southern_florida florida_settlement settlement_approv approv_miami miami_citi citi_commiss commiss_today today_effect effect_agreement agreement_sign sign_parti parti_resolv resolv_claim claim_stem stem_shoot shoot_offic offic_conduct conduct_violent violent_crime crime_control control_enforc enforc_find find_issu issu_juli juli_identifi identifi_pattern pattern_practic practic_excess excess_forc forc_shoot shoot_violat violat_fourth fourth_amend amend_constitut constitut_citi citi_complianc complianc_settlement settlement_monitor monitor_independ independ_review review_former former_tampa tampa_florida florida_polic polic_chief chief_jane jane_castor castor_settlement settlement_agreement agreement_citi citi_implement implement_comprehens comprehens_reform reform_ensur ensur_constitut constitut_polic polic_support support_public public_trust trust_settlement settlement_agreement agreement_design design_minim minim_shoot shoot_effect effect_quick quick_investig investig_shoot shoot_occur occur_measur measur_includ includ_settlement settlement_repres repres_renew renew_commit commit_citi citi_miami miami_chief chief_rodolfo rodolfo_llane llane_provid provid_constitut constitut_polic polic_miami miami_resid resid_protect protect_public public_safeti safeti_sustain sustain_reform reform_said said_princip princip_deputi deputi_general general_gupta gupta_agreement agreement_help help_strengthen strengthen_relationship relationship_communiti communiti_serv serv_improv improv_account account_offic offic_fire fire_weapon weapon_unlaw unlaw_provid provid_communiti communiti_particip particip_enforc enforc_today today_agreement agreement_result result_joint joint_effort effort_citi citi_miami miami_ensur ensur_miami miami_polic polic_continu continu_effort effort_make make_communiti communiti_safe safe_protect protect_sacr sacr_constitut constitut_citizen citizen_said said_ferrer ferrer_oversight oversight_communic communic_agreement agreement_seek seek_make make_perman perman_posit posit_chang chang_former former_chief chief_orosa orosa_chief chief_llane llane_made made_applaud applaud_citi citi_commiss commiss_settlement settlement_agreement agreement_build build_upon upon_import import_reform reform_implement implement_citi citi_sinc sinc_issu issu_find fin

C. Use the create_dtm function and the `processed_text_bigrams` column to create a document-term matrix (`dtm_bigram`) with these bigrams. Keep the following three columns in the data: `id`, `topics_clean`, and `compound` 

D. Print the (1) dimensions of the `dtm` matrix from question 2.2  and (2) the dimensions of the `dtm_bigram` matrix. Comment on why the bigram matrix has more dimensions than the unigram matrix 

E. Find and print the 10 most prevelant bigrams for each of the three topics_clean using the `get_topwords` function from 2.2

In [89]:
metadata = doj_subset_wscore[['id', 'topics_clean', 'compound']]

list_of_strings = doj_subset_wscore['processed_text_bigrams']

dtm_bigram = create_dtm(list_of_strings, metadata)

print("Unigram DTM dimensions:", dtm.shape)
print("Bigram DTM dimensions:", dtm_bigram.shape)


Unigram DTM dimensions: (717, 6760)
Bigram DTM dimensions: (1398, 71336)


In [47]:
dtm_bigram = dtm_bigram.apply(pd.to_numeric, errors='coerce')
print(dtm_bigram.dtypes)

dtm_bigram = dtm_bigram.fillna(0)

def get_topwords(dtm, n_topwords):
    top_words = {}
    for col in dtm.columns:
        top = dtm[col].nlargest(n_topwords).index.tolist()
        top_words[col] = top
    return top_words

# top words
top_bigrams_per_topic = get_topwords(dtm_bigram, 10)

#top bigrams
for topic, bigrams in top_bigrams_per_topic.items():
    print(f"Top 10 bigrams for {topic}: {bigrams}")

print(dtm_bigram.head())
print(dtm_bigram.dtypes)


index                  int64
id                   float64
topics_clean         float64
compound             float64
aaron_ford             int64
                      ...   
zone_varianc           int64
zunggeemog_noel        int64
zunggeemog_prompt      int64
zunggeemog_write       int64
zwengel_princeton      int64
Length: 71336, dtype: object


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [48]:
top_bigrams_per_topic = get_topwords(dtm_bigram, 10)
top_bigrams_per_topic

#double checked 

{'index': [275, 321, 239, 571, 272, 152, 259, 33, 49, 609],
 'id': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 'topics_clean': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 'compound': [694, 649, 663, 671, 698, 690, 650, 601, 644, 700],
 'aaron_ford': [307, 0, 1, 2, 3, 4, 5, 6, 7, 8],
 'aaron_latham': [39, 66, 287, 0, 1, 2, 3, 4, 5, 6],
 'aaron_mcgrath': [388, 0, 1, 2, 3, 4, 5, 6, 7, 8],
 'aaron_parrish': [361, 0, 1, 2, 3, 4, 5, 6, 7, 8],
 'aaron_polster': [211, 0, 1, 2, 3, 4, 5, 6, 7, 8],
 'aaron_rice': [206, 53, 63, 106, 29, 34, 69, 130, 213, 240],
 'abandon_children': [421, 0, 1, 2, 3, 4, 5, 6, 7, 8],
 'abbat_washington': [586, 0, 1, 2, 3, 4, 5, 6, 7, 8],
 'abbi_broughton': [83, 0, 1, 2, 3, 4, 5, 6, 7, 8],
 'abbott_act': [278, 0, 1, 2, 3, 4, 5, 6, 7, 8],
 'abbott_indict': [86, 0, 1, 2, 3, 4, 5, 6, 7, 8],
 'abdomen_area': [535, 0, 1, 2, 3, 4, 5, 6, 7, 8],
 'abdomen_larg': [103, 0, 1, 2, 3, 4, 5, 6, 7, 8],
 'abdomen_previous': [275, 0, 1, 2, 3, 4, 5, 6, 7, 8],
 'abduct_alert': [517, 0, 1, 2, 3, 4, 5, 6, 7, 8

# 4. Optional extra credit (2 points)

You notice that the pharmaceutical kickbacks press release we analyzed in question 1 was for an indictment, and that in the original data, there's not a clear label for whether a press release outlines an indictment (charging someone with a crime), a conviction (convicting them after that charge either via a settlement or trial), or a sentencing (how many years of prison or supervised release a defendant is sentenced to after their conviction).

You want to see if you can identify pairs of press releases where one press release is from one stage (e.g., indictment) and another is from a different stage (e.g., a sentencing).

You decide that one way to approach is to find the pairwise string similarity between each of the processed press releases in `doj_subset`. There are many ways to do this, so Google for some approaches, focusing on ones that work well for entire documents rather than small strings.

Find the top two pairs (so four press releases total)-- do they seem like different stages of the same crime or just press releases covering similar crimes?

In [80]:
def check_different_stages(text1, text2):
    stage_patterns = {
        'indictment': re.compile(r'\bindict\w*\b', re.IGNORECASE),
        'conviction': re.compile(r'\bconvict\w*\b', re.IGNORECASE),
        'sentencing': re.compile(r'\bsentenc\w*\b', re.IGNORECASE)
    }
    
    found_stages_text1 = {stage: bool(pattern.search(text1)) for stage, pattern in stage_patterns.items()}
    found_stages_text2 = {stage: bool(pattern.search(text2)) for stage, pattern in stage_patterns.items()}

    return any(found_stages_text1[stage] != found_stages_text2[stage] for stage in stage_patterns)


In [82]:
doj_subset_wscore['processed_text'] = doj_subset_wscore['processed_text'].fillna('')

tfidf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 3))
tfidf_matrix = tfidf_vectorizer.fit_transform(doj_subset_wscore['processed_text'])

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

def check_different_stages(text1, text2):
    stage_patterns = {
        'indictment': re.compile(r'\bindict\w*\b', re.IGNORECASE),
        'conviction': re.compile(r'\bconvict\w*\b', re.IGNORECASE),
        'sentencing': re.compile(r'\bsentenc\w*\b', re.IGNORECASE)
    }
    found_stages_text1 = {stage: bool(pattern.search(text1)) for stage, pattern in stage_patterns.items()}
    found_stages_text2 = {stage: bool(pattern.search(text2)) for stage, pattern in stage_patterns.items()}
    return any(found_stages_text1[stage] != found_stages_text2[stage] for stage in stage_patterns)

n = len(doj_subset_wscore)
top_pairs = []
for i in range(n):
    for j in range(i + 1, n):
        if cosine_sim[i, j] > 0.5:  
            if check_different_stages(doj_subset_wscore.iloc[i]['processed_text'], doj_subset_wscore.iloc[j]['processed_text']):
                top_pairs.append(((i, j), cosine_sim[i, j]))

top_pairs.sort(key=lambda x: x[1], reverse=True)
top_two_pairs = top_pairs[:2]

print(f"Total pairs identified: {len(top_pairs)}")
for pair, similarity in top_two_pairs:
    i, j = pair
    print(f"Similarity between Press Release {i} and Press Release {j}: {similarity:.4f}")
    print(f"Press Release {i} (Stage Check):\n{doj_subset_wscore.iloc[i]['processed_text']}\n")
    print(f"Press Release {j} (Stage Check):\n{doj_subset_wscore.iloc[j]['processed_text']}\n")

Total pairs identified: 77
Similarity between Press Release 530 and Press Release 555: 0.9442
Press Release 530 (Stage Check):
church hill maryland resid sentenc today year prison follow lifetim term supervis releas entic minor engag sexual activ attempt transfer obscen materi minor announc act general kenneth blanco crimin act benjamin greenberg southern florida robert moor plead guilti march judg daniel hurley southern florida moor employ secret assign white hous time arrest remain custodi sinc time moor sinc termin secret servic posit accord admiss made connect plea moor maintain profil social media applic provid platform exchang digit imag well voic text messag delawar state polic detect delawar child predat task forc creat profil site pose girl moor engag number onlin chat session mobil app period includ moor work number onlin chat moor undercov offic pose femal minor sexual natur sever occas moor sent pictur includ sexual explicit imag accord plea document arrest enforc discov mo

In [ ]:
#Press Release 530 and Press Release 555
# Press release 530 focuses on the sentencing details of an individual involved in
#enticing minors and distributing obscene material 
# Press release 555 focuses on the same individual pleading guilty to related charges 
# in federal court 

In [62]:
# Press Release 163 and Press Release on 173
# Press release 163 focuses on the guilty plea of an individual charged with 
#coercing minors into providing sexually explicit photos/videos 
# Press release 173 focuses on the sentencing of the same individual, highlighting 
# the prison term and supervised release following the guilty plea 